<h2 align="center"> SINGLE CELLS: </h2>
<h3 align="center"> UMA ANÁLISE DOS TIPOS DE CÉLULAS TRANSCRIPTÔMICAS COMPARTILHADAS E DISTINTAS EM ÁREAS NEOCORTICAIS </h3>

<hr>

<p> O neocórtex é uma região do cérebro com uma infinidade de tipos de células que são segregadas em camadas e áreas funcionalmente distintas atuando na percepção sensorial, cognição, geração de comandos motores, raciocínio espacial e linguagem. Na evolução dos mamíferos, o córtex sofreu maior expansão no número de células, camadas e áreas funcionais em comparação com o resto do cérebro, coincidindo com a aquisição de funções cognitivas cada vez mais sofisticadas, o objetivo deste trabalho é   </p>


<p> O projeto foi elaborado pelos estudantes durante o terceiro semestre de graduação da Ilum - Escola de Ciência, para a disciplina de Aprendizado de Máquina. O grupo é composto pelos estudantes: </p>

<p> Caio Eduardo Palatin  </p>
<p> João Pedro Aroucha de Brito  </p>
<p> Monyque Karoline de Paula Silva </p>
<p align="justify">

In [2]:
import pandas as pd
import scanpy as sc
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import random

In [ ]:
annotations = sc.read_h5ad('../complete_class/integrated.h5ad') #dont worry about this, get UMAP coords and labels
adata = sc.read_h5ad('../complete_class/combined.h5ad') #raw data